In [14]:
# IMPORTS

import json 
import os
from datetime import datetime
import yfinance as yf
from crewai import Agent, Task, Crew, Process
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults
import streamlit as st
from IPython.display import Markdown

In [2]:
# YAHOO FINANCE TOOL 
def fetch_stock_price(ticket):
    stock = yf.download(ticket, start="2023-08-08", end="2024-08-08")
    return stock

yahoo_finance_tool = Tool(
    name = "Yahoo Finance Tool",
    description = "Fetches stocks prices for {ticket} from the last year about a specific company from Yahoo Finance API",
    func = lambda ticket: fetch_stock_price(ticket)
)

In [3]:
# IMPORT OPENAI LLM - GPT
os.environ['OPENAI_API_KEY']=st.secrets['OpenAI_key']
llm = ChatOpenAI(model="gpt-3.5-turbo")

stockPriceAnalyst = Agent(
    role = "Senior stock price Analyst",
    goal = "Find the {ticket} stock price and analyses trends",
    backstory = """You're highly experienced in analyzing the price of an specific stock
    and make predictions about its future price.""",
    verbose = True,
    llm = llm,
    max_iter = 5,
    memory = True,
    tools = [yahoo_finance_tool],
    allow_delegation = False
)

In [4]:
# CREATE TASK STOCK_PRICE
getStockPrice = Task(
    description = "Analyze the stock {ticket} price history and create a trend analyses of up, down or sideways",
    expected_output = """Specify the current trend stock price - up, down or sideways. 
    eg. stock= 'APPL, price UP' """,
    agent = stockPriceAnalyst
)

In [5]:
# IMPORTANT SEARCH TOOL [DUCKDUCKGOSEARCHRESULTS]
search_tool = DuckDuckGoSearchResults(backend='news', num_results=10)

In [6]:
# CREATE AGENT NEWS_ANALYST
newsAnalyst = Agent(
    role = "Stock News Analyst",
    goal = """Create a short summary of the market news related to the stock {ticket} company. Specify the current trend - up, down or sideways with
    the news context. For each request stock asset, specify a numbet between 0 and 100, where 0 is extreme fear and 100 is extreme greed.""",
    backstory = """You're highly experienced in analyzing the market trends and news and have tracked assest for more then 10 years.

    You're also master level analyts in the tradicional markets and have deep understanding of human psychology.

    You understand news, theirs tittles and information, but you look at those with a health dose of skepticism. 
    You consider also the source of the news articles. """,
    verbose = True,
    llm = llm,
    max_iter = 10,
    memory = True,
    tools = [search_tool],
    allow_delegation = False
)

In [7]:
# CREATE TASK NEWS
get_news = Task(
    description = f"""Take the stock and always include BTC to it (if not request).
    Use the search tool to search each one individually. 

    The current date is {datetime.now()}.

    Compose the results into a helpfull report.""",
    expected_output = """A summary of the overall market and one sentence summary for each request asset. 
    Include a fear/greed score for each asset based on the news. Use format:
    <STOCK ASSET>
    <SUMMARY BASED ON NEWS>
    <TREND PREDICTION>
    <FEAR/GREED SCORE> """,
    agent = newsAnalyst
)

In [8]:
# CREATE STOCK ANALYST WRITER
stockAnalystWriter = Agent(
    role = "Senior Stock Analyts Writer",
    goal = """Analyze the trends price and news and write an insighfull compelling and informative 3 paragraph long newsletter based on the stock report and price trend. """,
    backstory = """You're widely accepted as the best stock analyst in the market. You understand complex concepts and create compelling stories
    and narratives that resonate with wider audiences. 

    You understand macro factors and combine multiple theories - eg. cycle theory and fundamental analyses. 
    You're able to hold multiple opinions when analyzing anything. """,
    verbose = True,
    llm = llm,
    max_iter = 5,
    memory = True,
    allow_delegation = True
)

In [9]:
# CREATE TASK TO WRITE ANALISYS
writeAnalyses = Task(
    description = """Use the stock price trend and the stock news report to create an analyses and write the newsletter about the {ticket} company
    that is brief and highlights the most important points.
    Focus on the stock price trend, news and fear/greed score. What are the near future considerations?
    Include the previous analyses of stock trend and news summary. """,
    expected_output = """An eloquent 3 paragraphs newsletter formated as markdown in an easy readable manner. It should contain:

    - 3 bullets executive summary 
    - Introduction - set the overall picture and spike up the interest
    - main part provides the meat of the analysis including the news summary and fead/greed scores
    - summary - key facts and concrete future trend prediction - up, down or sideways. """,
    agent = stockAnalystWriter,
    context = [getStockPrice, get_news]
)

In [10]:
# CREATE CREW
crew = Crew(
    agents = [stockPriceAnalyst, newsAnalyst, stockAnalystWriter],
    tasks = [getStockPrice, get_news, writeAnalyses],
    verbose = 2,
    process = Process.hierarchical,
    full_output = True,
    share_crew = False,
    manager_llm = llm,
    max_iter = 15
)

In [20]:
results = crew.kickoff(inputs={'ticket': 'NVDA'})

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Analyze the stock NVDA price history and create a trend analyses of up, down or sideways


> Entering new CrewAgentExecutor chain...
I need to gather information on the stock NVDA price history in order to create a trend analysis of whether it is going up, down, or sideways.

Action: Delegate work to co-worker
Action Input: {"coworker": "Senior Stock Analyts Writer", "task": "Analyze NVDA stock price history and determine trend analysis", "context": "I need you to analyze the historical stock price data of NVDA and determine whether the trend is currently up, down, or sideways. Please provide a detailed analysis with supporting evidence."}

> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: Based on my analysis of NVDA's historical stock price data, the trend appears to be upward. Looking at the price movements over the past few months, NVDA has shown consistent growth and has outperformed

In [24]:
list(results.keys())

['final_output', 'tasks_outputs']

In [23]:
Markdown(results['final_output'])

Error executing tool. Co-worker mentioned not found, it must be one of the following options:
- Senior stock price analyst
- Stock news analyst
- Senior stock analysts writer